In [ ]:
# тест-суит - Owasp-Benchmark Java

from os import replace
from bs4 import BeautifulSoup
import re

import csv
from google.colab import drive
import ast
import json


# Подключаем Google Диск
drive.mount('/content/drive')

# Ваш HTML-код
'''
html_code = """


"""
results = []
soup = BeautifulSoup(html_code, 'html.parser')

for li in soup.find_all('li'):
    result = []

    match = re.search(r'BenchmarkTest\w*', li.text)
    if match:
        benchmark_test_name = match.group(0)
        result.append(benchmark_test_name)
    else:
        print("Слово не найдено.")
        result.append("")  # Добавляем пустую строку, если слово не найдено

    if "Query built from user-controlled sources" in li.text or "Query built by concatenation with a possibly-untrusted string" in li.text:
        modified_text = "[89, 564]"
        result.append(modified_text)

    if "Uncontrolled data used in path expression" in li.text:
        modified_text = "[22, 23, 36, 73]"
        result.append(modified_text)
    if "Insecure randomness" in li.text:
         modified_text = "[330, 338]"
         result.append(modified_text)
    if "Use of a broken or risky cryptographic algorithm" in li.text or "Use of a potentially broken or risky cryptographic algorithm" in li.text:
        modified_text = "[327, 328]"
        result.append(modified_text)
    if "Trust boundary violation" in li.text:
        modified_text = "[501]"
        result.append(modified_text)

    if "high" in li.text.lower():
        result.append("high")

    if "critical" in li.text.lower():
        result.append("critical")

    results.append(result)


# Записываем результаты в CSV-файл
with open('/content/drive/My Drive/codeql_new.csv', 'a', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    for item in results:
        if(item!="None"):
          writer.writerow(item)

'''

# Загрузка данных из CSV файлов
formatted_issues_path = '/content/drive/My Drive/codeql_new.csv'
real_vulnerabilities_path = '/content/drive/My Drive/expectedresults-1.2.csv'  # Укажите путь к вашему файлу с реальными уязвимостями

# Попробуйте разные параметры
# Чтение реальных уязвимостей
with open(real_vulnerabilities_path, mode='r', encoding='utf-8') as real_vul_file:
    real_vulnerabilities = list(csv.reader(real_vul_file, delimiter=';'))
    # Убираем заголовки
    real_vulnerabilities = real_vulnerabilities[1:]  # Пропускаем первую строку

# Чтение форматированных проблем
with open(formatted_issues_path, mode='r', encoding='utf-8') as formatted_issues_file:
    formatted_iss = list(csv.reader(formatted_issues_file, delimiter=';'))
    # Убираем заголовки
    formatted_iss = formatted_iss[1:]  # Пропускаем первую строку

#formatted_issues = pd.DataFrame(formatted_iss)
formatted_issues = formatted_iss

# Вывод названий колонок и размера
print("Размер real_vulnerabilities:", len(real_vulnerabilities))

# Фильтрация реальных уязвимостей
# Инициализация списка для хранения реальных уязвимостей

# Цикл по всем записям в real_vulnerabilities
real_vulnerabilities_true = []  # Создаем пустой список для хранения соответствующих элементов

for real_issue in real_vulnerabilities:
    if 'true' in str(real_issue):  # Проверяем, содержится ли строка 'true' в элементе
      real_vulnerabilities_true.append(real_issue)

print(len(real_vulnerabilities_true))

# Инициализация счетчиков
TP = 0  # True Positives
FP = 0  # False Positives
FN = 0  # False Negatives

result = {}
negative = {}
for real_issue in real_vulnerabilities_true:
  cleaned_string = str(real_issue).strip("[]'")
  data_list = cleaned_string.split(',')
  benchmark_name = data_list[0].replace('.java', "")
  result[benchmark_name] = []
  negative[benchmark_name] = []

# Сравнение данных
for real_issue in real_vulnerabilities_true:
    # Удаление квадратных скобок и лишних пробелов
    cleaned_string = str(real_issue).strip("[]'")
    # Разделение строки на элементы списка
    data_list = cleaned_string.split(',')
    benchmark_name = data_list[0]
    cwe = data_list[3]
    for formatted_issue in formatted_issues:
      cleaned_string = str(formatted_issue)[1:-1]
      form_list = cleaned_string.split(',')
      file_name = str(re.findall(r'BenchmarkTest\d+', form_list[0])[0])
      cweStr = cleaned_string.replace(file_name+",","").replace("'","").replace(",high","").replace("[","").replace("]", "");
      if file_name==benchmark_name and cwe in cweStr:
        TP += 1

# Дополнительный блок для нахождения ложных отрицаний
for real_issue in real_vulnerabilities_true:
    cleaned_string = str(real_issue).strip("[]'")
    # Разделение строки на элементы списка
    data_list = cleaned_string.split(',')
    benchmark_name = data_list[0]
    cwe = data_list[3]

    # Проверяем, есть ли это реальное нарушение среди отформатированных
    if not any(

        (str(re.findall(r'BenchmarkTest\d+', str(formatted_issue)[1:-1].split(',')[0])[0])==benchmark_name and cwe in str(real_issue).strip("[]'").replace(str(re.findall(r'BenchmarkTest\d+', str(formatted_issue)[1:-1].split(',')[0])[0])+",","").replace("'","").replace(",high","").replace("[","").replace("]", ""))
        for formatted_issue in formatted_issues

    ):
        # Если не найдено в formatted_issues, добавляем к ложным отрицаниям
        FN += 1

for formatted_issue in formatted_issues:
    cleaned_string = str(formatted_issue)[1:-1]
    form_list = cleaned_string.split(',')
    file_name = str(re.findall(r'BenchmarkTest\d+', form_list[0])[0])
    cweStr = cleaned_string.replace(file_name+",","").replace("'","").replace(",high","").replace("[","").replace("]", "");
    if not any(
      (file_name in str(real_issue) and str(real_issue).strip("[]'").split(",")[3] in cweStr)
        for real_issue in real_vulnerabilities
    ):
        FP += 1


# Расчет метрик
recall = TP / (TP + FN)
precision = TP / (TP + FP)
f1_score = 2*(recall*precision)/(recall+precision)
# Вывод результатов
print(f"Recall: {recall:.2f}")
print(f"precision: {precision:.2f}")
print(f"F1-score: {f1_score:.2f}")



Mounted at /content/drive
Размер real_vulnerabilities: 2740
1415
Recall: 0.71
precision: 0.98
F1-score: 0.82


In [ ]:
# тест-суит - OWASP Security Shepherd


from os import replace
from bs4 import BeautifulSoup
import re
import csv
from google.colab import drive
import ast
import json


# Подключаем Google Диск
drive.mount('/content/drive')

# Ваш HTML-код
'''
html_code = """


"""


results = []
soup = BeautifulSoup(html_code, 'html.parser')

for li in soup.find_all('li'):
    result = []
    pattern = r"([a-zA-Z0-9_]+).java</a>:(\d+)"
    match = re.search(pattern, li.text)
    print(match)

    if match:
        filename = match.group(1) + ".java"  # Extract the filename (without .java)
        number = match.group(2)  # Extract the number
        result.append(filename)
        result.append(number)


    if "Server-side request forgery" in li.text:
        modified_text = "[918]"
        result.append(modified_text)
    if "Resolving XML external entity in user-controlled data" in li.text:
        modified_text = "[611,776,827]"
        result.append(modified_text)
    if "Using a static initialization vector for encryption" in li.text:
        modified_text = "[1204,329]"
        result.append(modified_text)

    if "Failure to use secure cookies" in li.text:
        modified_text = "[614]"
        result.append(modified_text)

    if "Cross-site scripting" in li.text:
        modified_text = "[79]"
        result.append(modified_text)

    if "Query built from user-controlled sources" in li.text or "Query built by concatenation with a possibly-untrusted string" in li.text:
        modified_text = "[89, 564]"
        result.append(modified_text)

    if "Uncontrolled data used in path expression" in li.text:
        modified_text = "[22, 23, 36, 73]"
        result.append(modified_text)
    if "Insecure randomness" in li.text:
         modified_text = "[330, 338]"
         result.append(modified_text)

    if "Use of a broken or risky cryptographic algorithm" in li.text or "Use of a potentially broken or risky cryptographic algorithm" in li.text:
        modified_text = "[327, 328]"
        result.append(modified_text)
    if "Trust boundary violation" in li.text:
        modified_text = "[501]"
        result.append(modified_text)
    if "Information exposure through an error message" in li.text:
        modified_text = "[209]"
        result.append(modified_text)

    if "high" in li.text.lower():
        result.append("high")

    if "critical" in li.text.lower():
        result.append("critical")

    if "medium" in li.text.lower():
        result.append("Medium")
    #print(result)

    results.append(result)
    result.clear()

#print(results)


# Записываем результаты в CSV-файл
with open('/content/drive/My Drive/codeql_shepherd.csv', 'a', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    for item in results:
        if(item!="None"):
          writer.writerow(item)


'''
def parse_custom_csv_line(line):
    # Убираем кавычки и разбиваем строку по запятой
    parts = []
    current_part = ''
    in_quotes = False

    for char in line:
        if char == '"':
            in_quotes = not in_quotes  # Меняем состояние, если встречаем кавычки
        elif char == ',' and not in_quotes:
            parts.append(current_part.strip())  # Добавляем текущую часть в список
            current_part = ''  # Сбрасываем текущую часть
            continue

        current_part += char  # Добавляем символ к текущей части

    parts.append(current_part.strip())  # Добавляем последнюю часть

    return parts

# Загрузка данных из CSV файлов
formatted_issues_path = '/content/sample_data/codeql2.csv'
real_vulnerabilities_path = '/content/drive/My Drive/output.csv'  # Укажите путь к вашему файлу с реальными уязвимостями
formatted_issues = []
# Чтение реальных уязвимостей
with open(real_vulnerabilities_path, mode='r', encoding='utf-8') as real_vul_file:
    real_vulnerabilities_true = list(csv.reader(real_vul_file, delimiter=';'))
    # Убираем заголовки
    real_vulnerabilities_true = real_vulnerabilities_true[1:]  # Пропускаем первую строку
print(real_vulnerabilities_true)
formatted_iss = []
# Чтение форматированных проблем
with open(formatted_issues_path, mode='r', encoding='utf-8') as formatted_issues_file:
    #reader = csv.DictReader(formatted_issues_file, delimiter='"')
    for line in formatted_issues_file:
      if("file_name" not in line ):
            # Удаляем символы новой строки и пробелы по краям
        cleaned_line = line.strip()
        formatted_issues.append(parse_custom_csv_line(cleaned_line))

# Вывод названий колонок и размера
print("Размер real_vulnerabilities_true:", len(real_vulnerabilities_true))

# Фильтрация реальных уязвимостей
# Инициализация списка для хранения реальных уязвимостей

# Цикл по всем записям в real_vulnerabilities

# Инициализация счетчиков
TP = 0  # True Positives
FP = 0  # False Positives
FN = 0  # False Negatives

result = {}
negative = {}
for real_issue in real_vulnerabilities_true:
  cleaned_string = str(real_issue).strip("[]'")
  data_list = cleaned_string.split(',')
  benchmark_name = data_list[0].replace('.java', "")
  result[benchmark_name] = []
  negative[benchmark_name] = []

# Сравнение данных
for real_issue in real_vulnerabilities_true:
    # Удаление квадратных скобок и лишних пробелов
    cleaned_string = str(real_issue).strip("[]'")
    # Разделение строки на элементы списка
    data_list = cleaned_string.split(',')
    benchmark_name = data_list[1]
    real_line = data_list[2]

    for formatted_issue in formatted_issues:
      file_name = formatted_issue[0]
      line = formatted_issue[1]
      cweList = formatted_issue[2].replace("\"","").replace("'","").replace("[","").replace("]","").replace("'","").split(",")
      formatted_cwe = [f'cwe:{item}' for item in cweList]
      matches = [item for item in formatted_cwe if item in str(real_issue)]
      if file_name in benchmark_name and len(matches)!=0:
      # and int(line) == int(real_line):
        TP += 1


# Дополнительный блок для нахождения ложных отрицаний
for real_issue in real_vulnerabilities_true:
    cleaned_string = str(real_issue).strip("[]'")
    # Разделение строки на элементы списка
    data_list = cleaned_string.split(',')
    benchmark_name = data_list[1]
    real_line = data_list[2]

    # Проверяем, есть ли это реальное нарушение среди отформатированных
    if not any(

        (formatted_issue[0] in str(real_issue) and
         len([item for item in [f'cwe:{item}' for item in formatted_issue[2].replace("\"","").replace("'","").replace("[","").replace("]","").replace("'","").split(",")] if item in str(real_issue)])>0)
         #and int(formatted_issue[1])==int(real_line))
        for formatted_issue in formatted_issues

    ):
        # Если не найдено в formatted_issues, добавляем к ложным отрицаниям
        FN += 1


for formatted_issue in formatted_issues:
    file_name = formatted_issue[0]
    line = formatted_issue[1]
    cweList = formatted_issue[2].replace("\"","").replace("'","").replace("[","").replace("]","").replace("'","").split(",")
    formatted_cwe = [f'cwe:{item}' for item in cweList]
    matches = [item for item in formatted_cwe if item in str(real_issue)]
    if not any(
      (file_name in str(real_issue) and len(matches)>0)
       #and int(str(real_issue).strip("[]'").split(",")[2])==line)
        for real_issue in real_vulnerabilities_true
    ):
        FP += 1





# Расчет метрик
recall = TP / (TP + FN)
precision = TP / (TP + FP)
f1_score = 2*(recall*precision)/(recall+precision)
# Вывод результатов
print(f"Recall: {recall:.2f}")
print(f"precision: {precision:.2f}")
print(f"F1-score: {f1_score:.2f}")